In [3]:
import pandas as pd

In [43]:
##pip install neo4j

Note: you may need to restart the kernel to use updated packages.


Data is already loaded in the database, we just have to connect to it to ask some queries

In [4]:
from neo4j import GraphDatabase

In [5]:
uri ="bolt://localhost:7687"
user = "neo4j"
password = "mda_2022"

In [6]:
##To see if the connection work
class HelloWorldExample:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def print_greeting(self, message):
        with self.driver.session() as session:
            result = session.run("CREATE (a:Greeting) "
                        "SET a.message = $message "
                        "RETURN a.message + ', from node ' + id(a)", message=message)
            name = result.single()[0]  
            print(name)
            
    def delete_greeting(self):
        with self.driver.session() as session:
            result = session.run("MATCH (a:Greeting)"
                        "DETACH DELETE (a)")
            print("done")
    
if __name__ == "__main__":
    greeter = HelloWorldExample(uri,user,password)
    greeter.print_greeting("hello world")
    greeter.delete_greeting()
    greeter.close()

hello world, from node 947
done


In [7]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(uri,auth=(user, password))
session = driver.session()
result = session.run("MATCH (s:Speech) RETURN s AS speech")
names = [record["speech"] for record in result]
session.close()
driver.close()

In [15]:
pd.DataFrame(names)

,city,content,count_commas,country,date,highest_speaker_count,location,pages,sentiment,sentimentScore,specific_location,state,title
0,New York,Hillary Clinton: Thank you very much. I have t...,2,USA,2013-09-24,10,Sheraton New York Hotel and Towers,19,NaN,NaN,Sheraton New York Hotel and Towers,New York,CGI_2013
1,Washington D.C.,"Well, good morning. Giving all praise and hono...",1,USA,2016-02-04,0,Washington Hilton,7,NaN,NaN,Washington Hilton,no_state,Prayer_Breakfast_2016
2,Chicago,"Good morning, everybody. I hope you all had a ...",1,USA,2008-12-01,0,no_specific_location,5,NaN,NaN,no_specific_location,Illinois,Security_Team_Announcement
3,Cairo,Thank you so much. Good afternoon. I am honore...,1,Egypt,2009-06-04,0,no_specific_location,14,NaN,NaN,no_specific_location,no_state,Cairo_University
4,Washington D.C.,There's been another mass shooting in America ...,0,USA,2015-10-01,0,no_specific_location,4,NaN,NaN,no_specific_location,no_state,Umpqua_Community_College_Shootings
...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,Washington D.C.,"Thank you, everybody, for being here. As we al...",0,USA,2010-01-14,0,White House,3,NaN,NaN,White House,no_state,Finance_Crisis_Fee
433,Buenos Aires,Hola! Thank you. Thank you so much. Muchas gra...,2,Argentina,2016-03-23,25,Usina Del Arte,22,NaN,NaN,Usina Del Arte,no_state,YLAI_Town_Hall
434,Washington D.C.,"Good evening. On Wednesday, 14 Americans were ...",1,USA,2015-12-06,0,White House,5,NaN,NaN,White House,no_state,Oval_Office_Counter_Terrorism_Agenda
435,New York,It is great to be here in The Big House -- and...,0,USA,2010-05-01,0,Clinton Global Initiative on Human Trafficking,8,NaN,NaN,Clinton Global Initiative on Human Trafficking,New York,Michigan_University_Commencement
